In [1]:
import os, tempfile
import numpy as np

from napari_stitcher import \
    (_reader,
     _registration,
     _fusion,
     _viewer_utils,
     _sample_data,
     _writer)

import dask.diagnostics

import napari
viewer = napari.Viewer()

In [2]:
# Load dataset from file

# filename = '/Users/malbert/software/napari-stitcher/image-datasets/arthur_20230613_e1_t0.czi'
# xims = _reader.read_mosaic_image_into_list_of_spatial_xarrays(filename)

# Test with generated dataset

xims = _sample_data.generate_tiled_dataset(
    ndim=2,
    N_t=50,
    N_c=1,
    tile_size=50,
    tiles_x=3,
    tiles_y=3,
    zoom=2,
    overlap=0,
    drift_scale=2,
    shift_scale=2,
)

In [3]:
layer_tuples = _viewer_utils.create_image_layer_tuples_from_xims(xims)
for lt in layer_tuples:
    viewer.add_image(lt[0], **lt[1])

In [4]:
tmpdir = tempfile.TemporaryDirectory()
with dask.diagnostics.ProgressBar():
    params = _registration.stabilize(xims, sigma=3)
    xfused = _fusion.fuse(xims, params, tmpdir=tmpdir)

[########################################] | 100% Completed | 3.03 sms
[########################################] | 100% Completed | 1.27 sms


In [5]:
ilts = _viewer_utils.create_image_layer_tuples_from_xims([xfused], name_prefix='fused')
for i in ilts:
    viewer.add_image(i[0], **i[1])

In [6]:
with dask.diagnostics.ProgressBar():
    _writer.save_xim_as_tif('fused.tif', xfused)

[########################################] | 100% Completed | 106.03 ms
